## 빅데이터 실습

### 제주도 핫플레이스 시각화

#### 데이터 불러오기

In [58]:
# 필요 라이브러리 사용등록
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import MarkerCluster #마커클러스터용 클래스 추가

In [2]:
# 추가 라이브러리
import requests
import time
from tqdm import tqdm

In [3]:
# 한글 깨짐 문제 해결
from matplotlib import rcParams, font_manager, rc

font_path  = 'C:/Windows/Fonts/NanumGothicCoding.ttf'
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)
rcParams['axes.unicode_minus'] = False

In [4]:
# 불필요한 경고메시지 제거
import warnings
warnings.filterwarnings('ignore')

In [5]:
# 데이터 불러오기
rawTotal = pd.read_excel('../day04/data/1_crawling_raw.xlsx')
rawTotal

,content,date,like,place,tags
0,더 늦기전에 제주도로 떠나쟈😍.#제주핫플레이스 간단히 1탄 모아봄!#제주여행 요 필...,2018-12-07,"1,402",NaN,"['#제주핫플레이스', '#제주여행', '#제주여행', '#제주도여행', '#제주가..."
1,12월에 제주도를 가야하는이유🍊.#제주핫플 모음 끝판왕😍이거 하나면 겨울 #제주여행...,2018-12-03,368,NaN,"['#제주핫플', '#제주여행', '#제주', '#제주도', '#제주도맛집', '#..."
2,11월 놓치지 말아야 할 제주 관광.가을바람이 최고조에 이르는 11월추운 겨울을 앞...,2018-11-02,166,NaN,"['#honestin', '#어니스틴', '#제주여행', '#제주', '#제주도',..."
3,국민학교세대#제주관광#제주살이#제주이주민#아라동주민#삼남매집 #새해첫날#드라이브#명...,2019-01-01,28,명월국민학교,"['#제주관광', '#제주살이', '#제주이주민', '#아라동주민', '#삼남매집'..."
4,#제주관광 #제주 #돔나이트 #스트레스 #풀자 #춤추며 #땀날려 #가끔은괜찮아 #인...,2019-01-01,12,NaN,"['#제주관광', '#제주', '#돔나이트', '#스트레스', '#풀자', '#춤추..."
...,...,...,...,...,...
8796,Now it's purple😆💜수색 넘 이쀼리 🤭 올해는 아퓨지말구 즐거운 일만 가...,2019-01-01,42,NaN,"['#일상', '#구기자', '#티타임', '#2019', '#새해', '#차스타그..."
8797,.간다간다간다#제주 #친정여행 #겨울방학 #제주여행앞머리는 괜히 전날 잘라서 또순이...,2019-01-01,50,NaN,"['#제주', '#친정여행', '#겨울방학', '#제주여행앞머리는', '#친정찬스'..."
8798,2019년 모두 복 많이많이 받고 풍요로운 한해 보내길 ♥️#사진좀찍으시는님덕에건져...,2019-01-01,68,Jeju,"['#사진좀찍으시는님덕에건져또', '#이순간']"
8799,2018.12.25 ⠀룸에 의자가 두개 밖에 없어서 ⠀이현이는 유모차에 앉아서 식사...,2019-01-01,70,NaN,[]


In [6]:
# 위치정보 확인
locCounts = rawTotal['place'].value_counts()

In [7]:
locCounts.head(3)

place
Jeju                    271
Jeju-do                 179
Jungle Book by Alice    108
Name: count, dtype: int64

In [8]:
# DF로 변경
dfLocCount = pd.DataFrame(locCounts)
dfLocCount.head(3)

,count
place,
Jeju,271
Jeju-do,179
Jungle Book by Alice,108


In [9]:
dfLocCount.to_excel('./data/3_location_counts.xlsx')

In [10]:
locations = list(dfLocCount.index)

In [11]:
locations[9]

'폼포코식당_pompokokitchen'

##### 카카오 OpenAPI로 장소 검색
- 카카오 개발자 사이트에서 애플리케이션 등록
- 키를 생성

In [12]:
from email import header


shopName = '제주크리스마스박물관'
## keyword는 가게명, address는 가게 주소로 검색
url = f'https://dapi.kakao.com/v2/local/search/keyword.json?query={shopName}'

header = {'Authorization' : 'KakaoAK 본인키'}

places = requests.get(url, headers=header).json()
places

UnicodeEncodeError: 'latin-1' codec can't encode characters in position 8-10: ordinal not in range(256)

In [25]:
## 장소 검색 함수 findPlace
from numpy import place

def findPlace(shopName):
    # 1. 검색 API 주소 변수
    url = f'https://dapi.kakao.com/v2/local/search/keyword.json?query={shopName}'
    # 2. header 구성
    header = {'Authorization' : 'KakaoAK 키주소'}
    # 3. API를 웹으로 요청, 결과를 리턴
    places = requests.get(url, headers=header).json()['documents']
    # 4. 필요 정보만 추출
    place = places[0]
    realName = place['place_name']
    lat = place['y'] # 위도(latitude)
    lng = place['x'] # 경도(longitude)
    phoneNum =place['phone']
    realAddress = place['address_name']

    return [shopName, realName, phoneNum, realAddress, lat, lng]


In [24]:
## 테스트
result = findPlace('우드노트')
result

['우드노트',
 '우드노트',
 '0507-1330-5352',
 '서울 송파구 풍납동 299-1',
 '37.5345598007673',
 '127.111818009785']

In [15]:
# location 리스트를 반복문으로 주소 가져오는 처리
locationResults = []
for loc in tqdm(locations):
    try:
        data = findPlace(loc)
        locationResults.append(data)
        time.sleep(0.3)
    except:
        pass

  0%|          | 0/1028 [00:00<?, ?it/s]

100%|██████████| 1028/1028 [10:40<00:00,  1.60it/s]


In [18]:
len(locationResults)

774

In [19]:
## list --> DataFrame 변경
dfLocationResult = pd.DataFrame(locationResults)

In [20]:
dfLocationResult.columns = ['초기이름', '실제이름', '전화번호', '주소', '위도', '경도']

In [23]:
dfLocationResult.to_excel('./data/3_locations.xlsx', index= False)

##### 인스타그램 크롤링 정보와 카카오 API로 주소 검색결과 정보 병합

In [26]:
dfLocCount

,count
place,
Jeju,271
Jeju-do,179
Jungle Book by Alice,108
Seogwipo,66
제주도 크리스마스 박물관,59
...,...
제주마당,1
형제섬,1
동복해녀식당,1


In [27]:
dfLocationResult.head()

,초기이름,실제이름,전화번호,주소,위도,경도
0,Jeju,R고기 in Jeju,064-744-6222,제주특별자치도 제주시 도두일동 2623-4,33.503587560005954,126.46542934219148
1,Seogwipo,서귀포잠수함,064-732-6060,제주특별자치도 서귀포시 서홍동 707-5,33.2393033784206,126.558616052674
2,제주도 크리스마스 박물관,바이나흐튼 크리스마스박물관,010-4602-7976,제주특별자치도 서귀포시 안덕면 서광리 456,33.29131209935493,126.32779866033343
3,할로비치,할로비치,,서울 강남구 신사동 511-7,37.5182108620338,127.020825988861
4,제주에인감귤밭,제주에인감귤밭,,제주특별자치도 서귀포시 호근동 693,33.25656069063887,126.5390139270361


In [29]:
# reset_index로 plasce를 일반 컬럼으로 변경
dfLocCount.reset_index(inplace=True)

In [31]:
dfLocCount.head()

,place,count
0,Jeju,271
1,Jeju-do,179
2,Jungle Book by Alice,108
3,Seogwipo,66
4,제주도 크리스마스 박물관,59


In [34]:
locationData = pd.merge(left=dfLocationResult, right=dfLocCount, how='inner', left_on= '초기이름', right_on= 'place')

In [37]:
condition = locationData['실제이름']== '제주동문시장'
locationData[condition]

,초기이름,실제이름,전화번호,주소,위도,경도,place,count
71,제주동문재래시장,제주동문시장,,제주특별자치도 제주시 일도일동 1148-2,33.51278396493313,126.52830338722843,제주동문재래시장,5
211,제주시 동문시장,제주동문시장,,제주특별자치도 제주시 일도일동 1148-2,33.51278396493313,126.52830338722843,제주시 동문시장,2
296,제주시동문시장,제주동문시장,,제주특별자치도 제주시 일도일동 1148-2,33.51278396493313,126.52830338722843,제주시동문시장,1
461,동문시장,제주동문시장,,제주특별자치도 제주시 일도일동 1148-2,33.51278396493313,126.52830338722843,동문시장,1


In [36]:
## 중복데이터 확인
locationData['실제이름'].value_counts()

실제이름
제주동문시장                4
올레길 7코스(서귀포-월평 올레)    4
세화해변                  3
에코랜드테마파크              3
오설록 티뮤지엄              3
                     ..
수제홍과일모찌 애월하귀점         1
대천해수욕장                1
도라지식당                 1
호박다방                  1
디스이즈핫                 1
Name: count, Length: 708, dtype: int64

In [39]:
## 중복데이터 pivot
locationDataFinal = locationData.pivot_table(values='count', index=['실제이름', '주소', '위도', '경도'], aggfunc= sum)

In [41]:
locationDataFinal.reset_index(inplace=True)

In [43]:
locationDataFinal.to_excel('./3_locations_final.xlsx')

In [44]:
locationDataFinal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709 entries, 0 to 708
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   실제이름    709 non-null    object
 1   주소      709 non-null    object
 2   위도      709 non-null    object
 3   경도      709 non-null    object
 4   count   709 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 27.8+ KB


In [45]:
## 위도, 경도가 object에서 float64로 변경
locationDataFinal = locationDataFinal.astype({'위도':'float64', '경도':'float64'})

In [47]:
locationDataFinal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709 entries, 0 to 708
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   실제이름    709 non-null    object 
 1   주소      709 non-null    object 
 2   위도      709 non-null    float64
 3   경도      709 non-null    float64
 4   count   709 non-null    int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 27.8+ KB


##### 지도 시각화
- folium 사용

In [56]:
locHanlaMt = [33.362500, 126.533694]
map = folium.Map(location= locHanlaMt, zoom_start= 11, tiles= 'OpenStreetMap')

for i in range(len(locationDataFinal)):
    name = locationDataFinal['실제이름'][i] # DF 컬럼이름 행값 == i
    count = int(locationDataFinal['count'][i])
    lat = float(locationDataFinal['위도'][i])
    lng = float(locationDataFinal['경도'][i])
    folium.CircleMarker(location=[lat,lng], radius=count, color='red', popup=name).add_to(map)
map

In [57]:
map.save('./data/3_jeju_hotplaces.html')

In [63]:
## 마커 클러스터
mapLocs = []
mapNames = []

for i in range(len(locationDataFinal)):
    data = locationDataFinal.iloc[i] # 행 하나씩 읽어옴
    mapLocs.append([float(data['위도']), float(data['경도'])])
    mapNames.append(data['실제이름'])

locHanlaMt = [33.362500, 126.533694]
map = folium.Map(location= locHanlaMt, zoom_start= 11, tiles= 'OpenStreetMap')

markerCluster = MarkerCluster(locations=mapLocs, popups=mapNames, name='제주 핫플레이스', overlay=True, control=True)
markerCluster.add_to(map)
folium.LayerControl().add_to(map)

map

In [64]:
map.save('./data/3_jeju_hotplaces_cluster.html')